In [17]:
import numpy as np
from torchvision import transforms
from PIL import Image
import tritonclient.http as httpclient
from tritonclient.utils import triton_to_np_dtype

def rn50_preprocess(img_path="./img1.jpg"):
    img = Image.open(img_path)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(img).numpy()

transformed_img = np.expand_dims(rn50_preprocess(), axis=0)
print(transformed_img.shape)
# Setup a connection with the Triton Inference Server.
triton_client = httpclient.InferenceServerClient(url="triton-direct-s3-route-triton-test.apps.nebula.sl")

# Specify the names of the input and output layer(s) of our model.
test_input = httpclient.InferInput("input", transformed_img.shape, datatype="FP32")
test_input.set_data_from_numpy(transformed_img, binary_data=True)

test_output = httpclient.InferRequestedOutput("output", binary_data=True, class_count=1000)

# Querying the server
results = triton_client.infer(model_name="resnet50", inputs=[test_input], outputs=[test_output])
test_output_fin = results.as_numpy('output')

print(test_output_fin)

(1, 3, 224, 224)
[b'12.474468:90' b'11.525706:92' b'9.660507:14' b'8.406357:136'
 b'8.220253:11' b'8.117573:94' b'8.007619:12' b'7.699606:95'
 b'7.478129:16' b'7.277445:88' b'6.826757:91' b'6.547328:96'
 b'6.510481:15' b'6.223680:10' b'5.619094:13' b'5.257725:989'
 b'4.977933:86' b'4.914546:93' b'4.744319:87' b'4.690239:42'
 b'4.651163:18' b'4.638570:313' b'4.624142:17' b'4.560015:325'
 b'4.518538:46' b'4.389143:31' b'4.353136:1' b'4.215613:952'
 b'4.186939:696' b'3.989321:40' b'3.958246:82' b'3.920278:320'
 b'3.910043:448' b'3.893525:89' b'3.831218:319' b'3.822076:951'
 b'3.820356:317' b'3.815914:957' b'3.768542:59' b'3.751775:47'
 b'3.723830:23' b'3.723201:21' b'3.607714:20' b'3.533411:128'
 b'3.526182:85' b'3.518313:7' b'3.516794:912' b'3.434794:301'
 b'3.434337:133' b'3.383423:131' b'3.314316:19' b'3.313550:316'
 b'3.145559:326' b'3.129328:304' b'3.096002:310' b'3.086322:988'
 b'3.025621:129' b'2.988876:949' b'2.973809:613' b'2.969127:97'
 b'2.964943:311' b'2.951941:883' b'2.928314

In [32]:
import json

class_file_path = "imagenet-labels.json"  # Expecting a path to the JSON file
with open(f"./{class_file_path}", 'r') as f:
    class_names = json.load(f)

top_class = test_output_fin[0]
top_class_data = top_class.decode('utf-8')
class_names[int(top_class_data.split(':')[1])]

'lorikeet'